In [16]:
import os
import pandas as pd
from pandas_geojson import to_geojson, write_geojson
from geopy.geocoders import GoogleV3
import geocoder
import json
from textwrap import shorten

In [17]:
df = pd.read_csv('Branded_projects_map_template_popup_pin - Sheet1.csv')

In [18]:
df = df.dropna(subset='full_address')
df = df.fillna(value='tk')

## Geocoder Setup

In [19]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [20]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Article Link Formatter

In [21]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2023/01...
1     <a href="https://therealdeal.com/miami/2022/10...
2     <a href="https://therealdeal.com/miami/2022/03...
3     <a href="https://therealdeal.com/miami/2021/11...
4     <a href="https://therealdeal.com/miami/2020/01...
5     <a href="https://therealdeal.com/miami/2021/11...
6     <a href="https://therealdeal.com/miami/2020/11...
7     <a href="https://therealdeal.com/miami/2022/11...
8     <a href="https://therealdeal.com/miami/2021/03...
9     <a href="https://therealdeal.com/miami/2021/11...
10    <a href="https://therealdeal.com/miami/2022/12...
11    <a href="https://therealdeal.com/miami/2022/09...
12    <a href="https://therealdeal.com/miami/2022/10...
13    <a href="https://therealdeal.com/issues_articl...
14    <a href="https://therealdeal.com/miami/2022/04...
15    <a href="https://therealdeal.com/miami/2021/02...
16    <a href="https://therealdeal.com/miami/2022/11...
17    <a href="https://therealdeal.com/miami/202

## Write to GeoJSON

In [22]:
geo_json = to_geojson(df=df, lat='lat', lon='lon',
                     properties=['full_address','project_name','developers','brokerage','description_link','brand','story_link'])
write_geojson(geo_json, filename='SoFla_branded_projects.geojson', indent=4)

In [23]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/SoFlaBrandedProjects_scrollytelling'

In [24]:
print(base_name)

https://trd-digital.github.io/trd-news-interactive-maps/SoFlaBrandedProjects_scrollytelling
